In [ ]:
import numpy as np
import pandas as pd
import os
import pickle

### Data loading helpers

In [ ]:
DMAS_NAMES = ['DMA_A', 'DMA_B', 'DMA_C', 'DMA_D', 'DMA_E', 'DMA_F', 'DMA_G', 'DMA_H', 'DMA_I', 'DMA_J']
WEEK_LEN = 24 * 7

In [ ]:
from eval.ensembler import load_data

In [ ]:
demand, weather = load_data()

def get_data(for_week):
    cur_demand = demand.iloc[:for_week * WEEK_LEN]
    cur_weather = weather.iloc[:(for_week+1) * WEEK_LEN]
    cur_demand_gt = demand.iloc[for_week * WEEK_LEN : (for_week+1) * WEEK_LEN]
    return cur_demand, cur_weather, cur_demand_gt

### Make ensembled forecast

In [ ]:
from eval.ensembler import WaterFuturesEnsembler, performance_indicators
from models.wavenet import wavenet_lin
from models.LGBM import lgbm_robust
from models.rolling_average_week import rolling_average_week
from models.TSMix import tsmix
from models.prototype_based import prototype_7
from models.pattern_regression import pattern_regression_daily
from models.autoregressives import autoreg_no_preprocess

In [ ]:
ensmebled_forecast = []

ensembled_pis = pd.DataFrame(
            index=pd.MultiIndex.from_tuples([(week,dma) for week in range(13, 77) for dma in DMAS_NAMES], names=['Test week', 'DMA']),
            columns=['PI1', 'PI2', 'PI3'],
            dtype=float
        )

for week in range(13, 77):
    print(f'Week {week}...')
    cur_demand, cur_weather, cur_demand_gt = get_data(for_week=week)
    ensembler = WaterFuturesEnsembler(cur_demand, cur_weather)
    # TODO Add models
    ensembler.add_model(wavenet_lin)
    ensembler.add_model(lgbm_robust)
    ensembler.add_model(rolling_average_week(4))
    ensembler.add_model(tsmix)
    ensembler.add_model(prototype_7)
    ensembler.add_model(pattern_regression_daily)
    ensembler.add_model(autoreg_no_preprocess)
    ensembler.run()

    ensmebled_forecast.append(ensembler.ensemble_forecast)

    ensembled_pis.loc[week] = performance_indicators(ensembler.ensemble_forecast.astype(float), cur_demand_gt)

forecast = pd.concat(ensmebled_forecast)


In [ ]:
results = {
    'performance_indicators': ensembled_pis,
    'forecast': forecast
}

# Save results to disk
file_path = 'wfe_results/EnsembleLast1.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(results, f)